In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
import yaml

In [ ]:
var_df = pd.read_csv('features.csv',skiprows = 1).dropna(subset = ['Variable Name'])
var_df.index=var_df['Variable Name']
var_df['Variable Type'].drop_duplicates()
var_df.head()

In [ ]:
feature_list = [r[1]['Variable Name'] for r in var_df.iterrows() if r[1]['feature'] == 1]
target = [r[1]['Variable Name'] for r in var_df.iterrows() if r[1]['target'] == 1]

In [ ]:
dtypes = {}
for r in var_df.iterrows():
    if r[1]['Variable Type'] == 'Categorical':
        dtypes[r[1]['Variable Name']] = 'category'
    else:
        dtypes[r[1]['Variable Name']] = 'float'
dtypes_features = {k:v for k,v in dtypes.items() if k in feature_list}
with open('dtypes_features.yaml','w') as dtf:
    yaml.dump(dtypes_features,dtf,default_flow_style=False)

In [ ]:
data = pd.read_csv('data.csv',dtype = dtypes)
data.isna().any().any()

In [ ]:
print(set(data.columns) == set(feature_list))
set(data.columns).symmetric_difference(set(feature_list))

In [ ]:
set(data.columns).difference(set(feature_list))

The symmetric difference of the data columns and feature list shows that MALE_MAR_or_WID needs to change to 'MALE_MAR_WID'.

In [ ]:
data.rename(columns = {'MALE_MAR_or_WID':'MALE_MAR_WID'},inplace = True)

Extract X and y variables. Make training and testing set for benchmarking.

In [ ]:
from sklearn.model_selection import train_test_split
X = data[feature_list]
y = data[target]
X_tr, X_te, y_tr, y_te = train_test_split(X,y,test_size= .2, random_state=10) #Set random state for consistent benchmarking
X_tr.to_csv('X_tr.csv',index = False)
X_te.to_csv('X_te.csv',index = False)
y_tr.to_csv('y_tr.csv', index = False)
y_te.to_csv('y_te.csv', index = False)

There are no NA, therefore no (immediate) reason to impute values.
# Logistic regression model
* 10-fold Cross validate with grid search. It is a small data set so random search/ bayesian hyperparameter optimization is overkill
* Transforms:
  * One hot encode categorical variables.
  * Binary variables should be scaled to 0 or 1.
  * Numeric will be made approximately normal using Yeo-Johnson transform (sklearn power transform) as a monotone transform making data closer N(0,1). This transform is capable of taking the log(x + 1)-transform when appropriate (for $ amount variables and other >= variable). 
  * FeatureUnion creates one transform for all columns by name.
 

In [ ]:

f = feature_list[0]
feature_types = {f:var_df.loc[f,'Variable Type'] for f in feature_list}


In [ ]:
# modules needed for logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, MinMaxScaler, KBinsDiscretizer, Binarizer, RobustScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import log_loss, make_scorer,  roc_auc_score
from sklearn.impute  import SimpleImputer
#

from sklearn import set_config
set_config(display = 'diagram')
col_transformers = []
for k, v in feature_types.items():
    if k == 'AMOUNT':
        continue
    if v == 'Numerical':
        col_transformers += [('power_'+k,PowerTransformer(),[k])]
    if v == 'Categorical':
        col_transformers += [('onehot_'+k,OneHotEncoder(),[k])]
    if v == 'Binary':
        col_transformers += [('binary_'+k,MinMaxScaler(),[k])]
col_transformers += [('scaler_'+'AMOUNT',RobustScaler(),['AMOUNT'])]
ColumnTransformer(col_transformers)



In [ ]:

def interact_last(X):
    return np.c_[X[:,:-5]*X[:,-5:-4],X[:,:-5]*X[:,-4:-3],X[:,:-5]*X[:,-3:-2],X[:,:-5]*X[:,-2:-1],X[:,:-5]*X[:,-1:]]
def func_feature_name(transformer, feature_names):
    return feature_names[:-1]



feature_transform.fit_transform(X_tr)
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.impute import SimpleImputer

interaction_transformer = FeatureUnion([('',FunctionTransformer(func = None,feature_names_out='one-to-one')),
                            ('AMT',Pipeline([('multamt',FunctionTransformer(func = interact_last,feature_names_out=func_feature_name)),
                                            ('impute0',SimpleImputer(strategy = 'mean',missing_values = 0)),
                                            ('scale',StandardScaler())]))])

feature_transformer = Pipeline([('transforms',ColumnTransformer(col_transformers,remainder = 'drop',verbose_feature_names_out=False))])

logistic_model = Pipeline([('feature_transformer',feature_transformer),
                           ('logistic_regression',LogisticRegression(penalty = 'elasticnet',solver = 'saga'))])


In [ ]:
logistic_model

In [ ]:
log_loss_scorer = make_scorer(log_loss,greater_is_better=False,needs_proba=True)
auc_scorer = make_scorer(roc_auc_score,greater_is_better=True,needs_proba=True)

In [ ]:
skf = StratifiedKFold(n_splits=10,shuffle=True, random_state= 10)
logistic_model_cv = GridSearchCV(logistic_model,
                                 param_grid = {'logistic_regression__C': np.logspace(-4,4,10), 'logistic_regression__l1_ratio': np.linspace(0,1,10)},
                                 verbose=True,
                                 
                                 scoring= ['neg_log_loss', 'roc_auc'],
                                 cv= skf,
                                 refit = 'neg_log_loss',
                                 n_jobs = -1
                                 )

In [ ]:
logistic_model_cv.fit(X_tr,y_tr)

In [ ]:
X_tr['DURATION'].min()

In [ ]:
from sklearn.utils import estimator_html_repr

with open("logistic_pipeline.html", "w", encoding='utf-8') as f:
    f.write(estimator_html_repr(logistic_model_cv))

In [ ]:
logistic_model_cv.best_params_

In [ ]:
logistic_model_cv.best_score_

In [ ]:
cv_results = pd.DataFrame.from_dict(logistic_model_cv.cv_results_)
cv_results.sort_values(by='rank_test_neg_log_loss').to_csv('logistic_regression_CV.csv')

In [ ]:
import joblib
joblib.dump(logistic_model_cv,'logistic_model.joblib')

## test score

In [ ]:
log_loss(y_pred=logistic_model_cv.predict_proba(X_te)[:,1],y_true=y_te)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_score=logistic_model_cv.predict_proba(X_te)[:,1],y_true=y_te)

# Gradient Boosted decision trees
* Pros: Do not need to scale date
* Cons: Difficult to interpret. Might not extrapolate to unseen data.

In [ ]:
# Modules needed for tree-ensemble
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
#


param_dist = {'base_estimator__max_depth': [2**i for i in range(5)],
              'base_estimator__num_iterations':[50,100,200,400],
              'base_estimator__reg_lambda':np.logspace(-3,3),
              'base_estimator__learning_rate':np.logspace(-2,-.5),
             'base_estimator__feature_fraction':[.25,.5,1.],
             'base_estimator__bagging_freq':[0,1,5,10],
              'base_estimator__bagging_fraction':[1.]
             }




gbdt_model_cv = RandomizedSearchCV(CalibratedClassifierCV(lgb.LGBMClassifier(boosting_type = 'dart',
                                                                            interaction_constraints = 
                                                                             [[i for i,_ in enumerate(X_tr.columns) if _ !='DURATION'],
                                                                              [i for i,_ in enumerate(X_tr.columns) if _ =='DURATION']],
                                                                             monotone_constraints_method = 'advanced',
                                                                            monotone_constraints = [int(x in ['AMOUNT','DURATION']) for x in X_tr.columns])),
                                   n_iter=64,
                                   param_distributions = param_dist, 
                                   cv = skf,scoring= ['neg_log_loss', 'roc_auc'],refit = 'neg_log_loss')

In [ ]:
gbdt_model_cv.fit(X_tr,y_tr)


In [ ]:
gbdt_model_cv

In [ ]:
cv_results_gbdt = pd.DataFrame.from_dict(gbdt_model_cv.cv_results_)
cv_results_gbdt.sort_values(by='rank_test_neg_log_loss').to_csv('gbdt_CV.csv')

In [ ]:
log_loss(y_pred=gbdt_model_cv.predict_proba(X_te)[:,1],y_true=y_te)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_score=gbdt_model_cv.predict_proba(X_te)[:,1],y_true=y_te)

In [ ]:
joblib.dump(gbdt_model_cv,'gbdt_model.joblib')

In [ ]:
with open("gbdt_pipeline.html", "w", encoding='utf-8') as f:
    f.write(estimator_html_repr(gbdt_model_cv))

# Random forest
Do not use. One tree ensemble model is enough for now.

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
param_dist = {'base_estimator__max_depth': range(1,10),
              'base_estimator__num_iterations':[500],
              'base_estimator__reg_lambda':np.logspace(-3,3),
             'base_estimator__feature_fraction_bynode':[.2],
              'base_estimator__bagging_freq':[1],
             'base_estimator__bagging_fraction':[.9]}


from sklearn.calibration import CalibratedClassifierCV

rf_model_cv = RandomizedSearchCV(CalibratedClassifierCV(lgb.LGBMClassifier(boosting_type = 'rf')),
                                   n_iter=64,
                                   param_distributions = param_dist, 
                                   cv = skf,scoring =['neg_log_loss', 'roc_auc'],refit = 'neg_log_loss')

In [ ]:
rf_model_cv.fit(X_tr,y_tr)


In [ ]:
rf_model_cv

In [ ]:
cv_results_rf = pd.DataFrame.from_dict(rf_model_cv.cv_results_)
cv_results_rf.sort_values(by='rank_test_score').to_csv('rf_CV.csv')

In [ ]:
log_loss(y_pred=rf_model_cv.predict_proba(X_te)[:,1],y_true=y_te)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_score=rf_model_cv.predict_proba(X_te)[:,1],y_true=y_te)

In [ ]:
y_te.sum()